In [5]:
# DOCS: https://docs.microsoft.com/en-us/machine-learning-server/python-reference/revoscalepy/rxsqlserverdata
# DOCS: https://blogs.msdn.microsoft.com/mlserver/2018/07/10/run-r-and-python-remotely-in-sql-server-from-jupyter-notebooks-or-any-ide/

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from revoscalepy import RxComputeContext, RxInSqlServer, RxSqlServerData
from revoscalepy import rx_import

# Connection string to your SQL Server instance
conn_str = 'Driver=SQL Server;Server=DEVNOTEBOOK\MSSQLSERVER01;Database=Parcel56;Trusted_Connection=True;'

# Define the columns you will import
column_info = {
         "ParcelId" : { "type" : "integer" },
         "Swis" : { "type" : "integer" },
         "TotalAV" : { "type" : "integer" },                  
         "Acres" : { "type" : "integer" },         
         "Zip" : { "type" : "integer" }
     }

# Get the data from the SQL Server table
data_source = RxSqlServerData(table="dbo.CleanedAssessmentData", connection_string=conn_str, column_info=column_info)

computeContext = RxInSqlServer(
     connection_string = conn_str,
     num_tasks = 1,
     auto_cleanup = False
)

RxInSqlServer(connection_string=conn_str, num_tasks=1, auto_cleanup=False)

# import data source and convert to pandas dataframe
df = pd.DataFrame(rx_import(input_data = data_source))

print("Data frame:", df)

# Get all the columns from the dataframe.
columns = df.columns.tolist()

# Store the variable we'll be predicting on.
target = "TotalAV"

# Generate the training set.  Set random_state to be able to replicate results.
train = df.sample(frac=0.8, random_state=1)

# Select anything not in the training set and put it in the testing set.
test = df.loc[~df.index.isin(train.index)]

# Print the shapes of both sets.
print("Training set shape:", train.shape)
print("Testing set shape:", test.shape)

# Initialize the model class.
lin_model = LinearRegression()

# Fit the model to the training data.
lin_model.fit(train[columns], train[target])

# Generate our predictions for the test set.
lin_predictions = lin_model.predict(test[columns])
print("Predictions:", lin_predictions)

# Compute error between our test predictions and the actual values.
lin_mse = mean_squared_error(lin_predictions, test[target])
print("Computed error:", lin_mse)


Rows Read: 60903, Total Rows Processed: 60903, Total Chunk Time: 0.126 seconds 
Data frame:        ParcelId    Swis  TotalAV  Acres        Zip
0           297  562001    74000      0      14009
1           298  562001    82000   2180      14525
2           302  562001   105000      0      14423
3           303  562001    85000      0      14822
4           303  562001    85000    162      14569
5           304  562001    95000  11375      14113
6           305  562001   125000      0      14525
7           305  562001   125000    570      14214
8           306  562001    85000    104      14525
9           306  562001    85000   2540      14113
10          307  562001    81000   1914      14082
11          308  562001    65000   4221      14525
12          308  562001    65000   3444      14113
13          308  562001    65000      0      14530
14          310  562001    75000    126      14525
15          310  562001    75000    109      14113
16          310  562001    75000      0  